In [1]:
from imaging_functions import *
from aicsimageio import AICSImage
import os
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from skimage.io import imread
from skimage.measure import regionprops
from skimage.color import label2rgb
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
from skimage import exposure
from skimage.transform import rescale
from cellpose import plot
from skimage.measure import regionprops
from skimage.exposure import equalize_adapthist
from skimage import img_as_ubyte
device = torch.device("cuda:0")
import tifffile
from tqdm import tqdm

03-Jul-25 11:00:12 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).




Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.5 
platform:       	linux 
python version: 	3.10.13 
torch version:  	2.5.1! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [2]:
# === Parameters ===
input_dir = '/home/mars/Projects/mel_summer/experimental_data/outputs_rgb/s54/enhanced_rgb_frames/'
output_dir = '/home/mars/Projects/mel_summer/experimental_data/s54_cellpose_outputs'
scale_factor = 1.0  # Adjust so nuclei are ~30 px
os.makedirs(output_dir, exist_ok=True)

# === Collect image files ===
image_files = sorted([
    fname for fname in os.listdir(input_dir)
    if fname.lower().endswith(('.png', '.jpg', '.tif', '.tiff'))
])

# === Initialize Cellpose model ===
model = models.CellposeModel(device=device)

In [3]:
from skimage.io import imread, imsave
import tifffile


In [4]:
# === Process all frames with tqdm ===
for fname in tqdm(image_files, desc="Segmenting frames with Cellpose"):
    image_path = os.path.join(input_dir, fname)
    base_name = os.path.splitext(fname)[0]

    # === Load and normalize image ===
    img_rgb = imread(image_path).astype(np.float32) / 255.0

    # === Extract and enhance red channel ===
    red_channel = img_rgb[..., 0]
    red_eq = equalize_adapthist(red_channel, clip_limit=0.01)

    # === Blend channels and rescale ===
    blended = 0.8 * red_eq + 4.0 * img_rgb.mean(axis=-1)
    channel_img = np.clip(blended, 0, 1)
    scaled_img = rescale(channel_img, scale=scale_factor, preserve_range=True, anti_aliasing=True)
    scaled_img = img_as_ubyte(scaled_img)

    # === Run Cellpose ===
    masks_list, flows_list, _ = model.eval(
        [scaled_img],
        diameter=None,
        augment=True,
        tile_overlap=0.5,
        progress=False,
        cellprob_threshold=-4.0,
        flow_threshold=0.0,
    )

    masks = masks_list[0]
    flow_rgb   = flows_list[0][0]  # (H, W, 3)
    #flow_yx    = flows_list[0][1]  # (2, H, W)
    #cellprob   = flows_list[0][2]  # (H, W)
    
    # === Save mask ===
    imsave(os.path.join(output_dir, f'{base_name}_mask.tif'), masks.astype(np.uint16))
    
    # === Stack and save flow components ===
    #cellprob = cellprob[None, ...]  # (1, H, W)
    #flow_stack = np.concatenate([flow_yx, cellprob], axis=0).astype(np.float32)
    
    # tifffile.imwrite(
    #     os.path.join(output_dir, f'{base_name}_flows.tif'),
    #     flow_stack,
    #     imagej=True
    # )
    
    # === Save RGB flow visualization ===
    imsave(os.path.join(output_dir, f'{base_name}_flow_rgb.png'), flow_rgb)

Segmenting frames with Cellpose:   0%|                | 0/289 [00:00<?, ?it/s]/tmp/ipykernel_100290/1352564087.py:36: UserWarning: /home/mars/Projects/mel_summer/experimental_data/s54_cellpose_outputs/frame_000_mask.tif is a low contrast image
  imsave(os.path.join(output_dir, f'{base_name}_mask.tif'), masks.astype(np.uint16))
Segmenting frames with Cellpose:   0%|    | 1/289 [03:14<15:32:33, 194.28s/it]


KeyboardInterrupt: 

In [2]:
#Example terminal commands to run segment_parallel.py, which has all of the above code:


python segment_parallel.py \
  --input_dir /home/mars//Projects/mel_summer/experimental_data/outputs_rgb/s54/enhanced_rgb_frames \
  --output_dir /home/mars/Projects/mel_summer/experimental_data/s54_cellpose_outputs \
  --start 0 --stride 2 &


sleep 20


python segment_parallel.py \
   --input_dir /home/mars//Projects/mel_summer/experimental_data/outputs_rgb/s54/enhanced_rgb_frames \
  --output_dir /home/mars/Projects/mel_summer/experimental_data/s54_cellpose_outputs \
  --start 1 --stride 2 &


In [ ]:
python cellpose_gen.py \
--beg 28 \
--end 42 \
--start 0 --stride 2 &

sleep 20

python cellpose_gen.py \
--beg 28 \
--end 42 \
--start 1 --stride 2 &

python cellpose_gen.py \
--beg 42 \
--end 55 \
--start 0 --stride 2 &

sleep 20

python cellpose_gen.py \
--beg 42 \
--end 55 \
--start 1 --stride 2 &